# Show images from IMDB dataset for labelling and cleaning

Source: https://stackoverflow.com/questions/19471814/display-multiple-images-in-one-ipython-notebook-cell

In [ ]:
#use the native Tk backend
%matplotlib tk
import matplotlib.pyplot as plt

import os
import numpy as np
import pandas as pd
import PIL

import pyperclip
#sudo pip3 install pyperclip

In [ ]:
image_path = '/home/krittametht/dataset/agegender_imdb/annotations/gender/train/f'

In [ ]:
filenames = ! ls $image_path
filenames_df = pd.DataFrame(filenames)
print("Total # files:", len(filenames))

# sort according to int image id from filename format: ""%d.jpg"
sorted_index = filenames_df[0].str.rsplit('.').str[0].astype(int).sort_values().index
filenames_df = filenames_df.reindex(index=sorted_index)
filenames = filenames_df[0].tolist()

In [ ]:
def index_of(filenames_series, filename):
    return filenames_series.str.startswith(filename + ".").tolist().index(True)

In [ ]:
# to_filename = '46613'
# to_filename = '60334'
to_filename = '79459'
count = index_of(filenames_df[0], to_filename) + 1
print("count up to {}: ".format(to_filename), count)

In [ ]:
filenames_df.head(10)
# filenames_df.tail(10)

In [ ]:
def loop_each_n_items(l, n, start_index=0, end_index=0):
    end_i = end_index
    if end_index == 0:
        if start_index == 0:
            end_i = len(l)-n+1
        else:
            end_i = start_index+n
    for i in range(start_index,end_i,n):
        yield l[i:i+n]

In [ ]:
def onclick(event):
    if event.inaxes is not None:
        # the axes object on which the user clicked
        ax = event.inaxes
        # can use ax.children() to find which img artist is in this axes and extract the data from it

        subplot_title = ax.get_title()

        # copy subplot's title to clipboard
        pyperclip.copy(subplot_title.split(".")[0])

In [ ]:
# Source: https://nbviewer.jupyter.org/gist/minrk/7076095

def image_grid_plot(list_filenames, figsize=(20,20)):
    for sel_filenames in list_filenames:
        images = [ PIL.Image.open(f) for f in [os.path.join(image_path, fn) for fn in sel_filenames] ]

    fig = plt.figure(figsize=figsize)
    columns = 5
    for i, image in enumerate(images):
        sub = plt.subplot(len(images) / columns + 1, columns, i + 1)
        sub.set_title(image.filename.split('/')[-1])
        plt.imshow(image)
    return fig

In [ ]:
start_index_n = 60462
start_index = index_of(filenames_df[0], str(start_index_n))

plot_batch_size = 20
figsize = (20,20)

fig = image_grid_plot(loop_each_n_items(filenames, plot_batch_size, start_index, start_index+plot_batch_size), figsize=figsize)
cid = fig.canvas.mpl_connect('button_press_event', onclick)